# Dowload lyrics

In [1]:
#Assign your Genius.com credentials and select your artist
import lyricsgenius as genius
token = "CRG5jc-SgTRDpuJzU2neM_SajlResplv7DMi9jn1LmlNA-WiTvn2XseA_4rWb6y8"
artist_name = "JuL"
geniusCreds = f"{token}"
artist_name = f"{artist_name}"

In [2]:
Genius = genius.Genius(geniusCreds)
artist = Genius.search_artist(artist_name, max_songs=30)


Searching for songs by JuL...

Song 1: "J’oublie tout"
Song 2: "Tchikita"
Song 3: "Tu la love"
Song 4: "Dans ma paranoïa"
Song 5: "My World"
Song 6: "Wesh alors"
Song 7: "Sors le cross volé"
Song 8: "Lova"
Song 9: "Comme d’hab"
Song 10: "Dans mon dél"
Song 11: "Tout seul"
Song 12: "En Y"
Song 13: "Toto et Ninetta"
Song 14: "Au quartier"
Song 15: "Amnésia"
Song 16: "Mon bijou"
Song 17: "Mon amour"
Song 18: "Ma jolie"
Song 19: "Le sang"
Song 20: "Mama"
Song 21: "Ils sont jaloux"
Song 22: "Encore des paroles"
Song 23: "Malade"
Song 24: "On m’appelle l’ovni"
Song 25: "Briganté"
Song 26: "La classe"
Song 27: "Señora"
Song 28: "T’es pas le seul"
Song 29: "Pourquoi tu me fais le gros"
Song 30: "Coucou"

Reached user-specified song limit (30).
Done. Found 30 songs.


In [7]:
artist.save_lyrics()

Lyrics_JuL.json already exists. Overwrite?
(y/n): y
Wrote Lyrics_JuL.json.


In [8]:
import json
  
# Opening JSON file
f = open(f"Lyrics_{artist_name}.json")
  
# returns JSON object as 
# a dictionary
Artist = json.load(f)


In [9]:
#Create an empty dictionary to store your songs and related data
artist_dict = {}
def collectSongData(adic):
    dps = list()
    title = adic['title'] #song title
    url = adic['url'] #spotify url
    artist = adic['artist'] #artist name(s)
    song_id = adic['id'] #spotify id
    lyrics = adic['lyrics'] #song lyrics
    upload_date = adic['description_annotation']['annotatable']['client_timestamps']['lyrics_updated_at'] #lyrics upload date
    annotations = adic['annotation_count'] #total no. of annotations
    descr = adic['description'] #song descriptions
    
    dps.append((title,url,artist,song_id,lyrics,upload_date,annotations,descr)) #append all to one tuple list
    artist_dict[title] = dps #assign list to song dictionary entry named after song title
    
for song in Artist['songs']: 
    collectSongData(song) #check function works

In [10]:
artist_dict.keys()

dict_keys(['J’oublie tout', 'Tchikita', 'Tu la love', 'Dans ma paranoïa', 'My World', 'Wesh alors', 'Sors le cross volé', 'Lova', 'Comme d’hab', 'Dans mon dél', 'Tout seul', 'En Y', 'Toto et Ninetta', 'Au quartier', 'Amnésia', 'Mon bijou', 'Mon amour', 'Ma jolie', 'Le sang', 'Mama', 'Ils sont jaloux', 'Encore des paroles', 'Malade', 'On m’appelle l’ovni', 'Briganté', 'La classe', 'Señora', 'T’es pas le seul', 'Pourquoi tu me fais le gros', 'Coucou'])

In [11]:
import csv

def updateCSV_file():
    upload_count = 0 #Set upload counter
    location = f"./" #Pick file location
    print("input filename of song file, please add .csv")
    filename = input() #give your file a name
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file: #open a new csv file
        a = csv.writer(file, delimiter=',') #split by comma
        #(title,url,artist,song_id,lyrics,year,upload_date,annotations,descr)
        headers = ["Title","URL","Artist", "Song ID", "Lyrics", "Upload Date", "Annotations","Description"] #create header row
        a.writerow(headers) #add header row
        for song in artist_dict:
            a.writerow(artist_dict[song][0])
            upload_count+=1
            
        print(str(upload_count) + " songs have been uploaded")
updateCSV_file()

input filename of song file, please add .csv
julsong.csv
30 songs have been uploaded


# ML

In [2]:
import io
import os
import sys
import string
import numpy as np
import pandas as pd
from tensorflow import keras
from __future__ import print_function
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
translator = str.maketrans('', '', string.punctuation)

In [3]:
df = pd.read_csv('./julsong.csv')

In [4]:
def split_text(x):
    text = x['Lyrics']
    sections = text.split('\n\n')
    keys = {'Couplet 1': np.nan,'Couplet 2':np.nan,'Couplet 3':np.nan,'Couplet 4':np.nan, 'Refrain':np.nan}
    
    lyrics = str()

    single_text = []

    res = {}

    for s in sections:

        key = s[s.find('[') + 1:s.find(']')].strip()
        
        if ':' in key:
            key = key[:key.find(':')]

        if key in keys:
            single_text += [x.lower().replace('(','').replace(')','').translate(translator) for x in s[s.find(']')+1:].split('\n') if len(x) > 1]
        

        res['single_text'] =  ' \n '.join(single_text)
    return pd.Series(res)

df = df.join(df.apply(split_text, axis=1))


In [5]:
# Compute word frequencies
text_as_list = []

frequencies = {}

uncommon_words = set()

MIN_FREQUENCY = 7

MIN_SEQ = 5

BATCH_SIZE =  32


def extract_text(text):

    global text_as_list

    text_as_list += [w for w in text.split(' ') if w.strip() != '' or w == '\n']

df['single_text'].apply( extract_text )
print('Total words: ', len(text_as_list))


for w in text_as_list:
    frequencies[w] = frequencies.get(w, 0) + 1

uncommon_words = set([key for key in frequencies.keys() if frequencies[key] < MIN_FREQUENCY])
words = sorted(set([key for key in frequencies.keys() if frequencies[key] >= MIN_FREQUENCY]))


num_words = len(words)

word_indices = dict((w, i) for i, w in enumerate(words))

indices_word = dict((i, w) for i, w in enumerate(words))

print('Words with less than {} appearances: {}'.format( MIN_FREQUENCY, len(uncommon_words)))

print('Words with more than {} appearances: {}'.format( MIN_FREQUENCY, len(words)))


valid_seqs = []

end_seq_words = []

for i in range(len(text_as_list) - MIN_SEQ ):

    end_slice = i + MIN_SEQ + 1

    if len( set(text_as_list[i:end_slice]).intersection(uncommon_words) ) == 0:

        valid_seqs.append(text_as_list[i: i + MIN_SEQ])

        end_seq_words.append(text_as_list[i + MIN_SEQ])


print('Valid sequences of size {}: {}'.format(MIN_SEQ, len(valid_seqs)))


X_train, X_test, y_train, y_test = train_test_split(valid_seqs, end_seq_words, test_size=0.02, random_state=42)

print(X_train[2:5])

Total words:  9499
Words with less than 7 appearances: 1436
Words with more than 7 appearances: 232
Valid sequences of size 5: 2546
[['de', 'toi', '\n', 'elle', 'taime'], ['et', 'la', 'nuit', 'elle', 'rêve'], ['sang', '\n', 'jferai', 'tout', 'pour']]


In [6]:
# Data generator for fit and evaluate

def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, MIN_SEQ), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y


# Functions from keras-team/keras/blob/master/examples/lstm_text_generation.py

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)

    preds = exp_preds / np.sum(exp_preds)

    probas = np.random.multinomial(1, preds, 1)

    return np.argmax(probas)


def on_epoch_end(epoch, logs):

    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)


    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(X_train+X_test))

    seed = (X_train+X_test)[seed_index]


    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed

        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')

        examples_file.write(' '.join(sentence))


        for i in range(50):

            x_pred = np.zeros((1, MIN_SEQ))

            for t, word in enumerate(sentence):
                x_pred[0, t] = word_indices[word]


            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            sentence = sentence[1:]
            sentence.append(next_word)
            examples_file.write(" "+next_word)

        examples_file.write('\n')

    examples_file.write('='*80 + '\n')

    examples_file.flush()

In [7]:
def get_model():
    print('Build model...')
    model = Sequential()
    model.add(Embedding(input_dim=len(words), output_dim=1024))
    model.add(Bidirectional(LSTM(128)))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    return model

In [ ]:
model = get_model()

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
            "loss{loss:.4f}-acc{accuracy:.4f}-val_loss{val_loss:.4f}-val_acc{val_accuracy:.4f}" % \
            (len(words), MIN_SEQ, MIN_FREQUENCY)


checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', save_best_only=True)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)

callbacks_list = [checkpoint, print_callback, early_stopping]


examples_file = open('examples.txt', "w")

model.fit(generator(X_train, y_train, BATCH_SIZE),
                    steps_per_epoch=int(len(valid_seqs) / BATCH_SIZE) + 1,
                    epochs=20,
                    callbacks=callbacks_list,
                    validation_data=generator(X_test, y_train, BATCH_SIZE),
                    validation_steps=int(len(y_train) / BATCH_SIZE) + 1)

Build model...


2022-08-18 17:37:12.776510: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-18 17:37:12.776542: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (marthes44): /proc/driver/nvidia/version does not exist
2022-08-18 17:37:12.777462: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
79/80 [============================>.] - ETA: 0s - loss: 4.4334 - accuracy: 0.1578

INFO:tensorflow:Assets written to: ./checkpoints/LSTM_LYRICS-epoch001-words232-sequence5-minfreq7-loss4.4135-acc0.1613-val_loss4.8434-val_acc0.1178/assets


INFO:tensorflow:Assets written to: ./checkpoints/LSTM_LYRICS-epoch001-words232-sequence5-minfreq7-loss4.4135-acc0.1613-val_loss4.8434-val_acc0.1178/assets


80/80 [==============================] - 31s 346ms/step - loss: 4.4135 - accuracy: 0.1613 - val_loss: 4.8434 - val_accuracy: 0.1178
Epoch 2/20
27/80 [=========>....................] - ETA: 1s - loss: 2.8010 - accuracy: 0.3704